In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime, timedelta

In [ ]:
columns_not_needed_70 = ['user_id', 'account_age_in_days', 'bot', 'created_at', 'name', 'lang', 
                        'avg_tweets_per_actual_day', 'avt_favorite_count', 'total_replies',
                        'tweet_num_replies_ratio', 'max_number_of_tweet_in_a_day', 'total_likes',
                        'total_retweet_count', 'tweet_num_likes_ratio', 'number_of_tweets', 
                        'avg_text_length', 'statuses_count']
columns_not_needed_80 = ['user_id', 'account_age_in_days', 'bot', 'created_at', 'name', 'lang', 
                        'avg_tweets_per_actual_day', 'avg_text_length', 'max_number_of_tweet_in_a_day',
                        'number_of_tweets', 'total_likes', 'total_replies', 'tweet_num_replies_ratio',
                        'statuses_count']